# Homework 4: Basically bioinformatics

---
## Topic areas
* Functions
* I/O operations
* Dictionary lookups
* Data structures
* Control structures

---
## Introduction

Bioinformatics is a special field that blends **biology**, **mathematics/statistics**, and **computer science**. One note that is often left off is that the computer science that is done is often in the form of _Big Data_ computer science. 

One reason many computer science classes in the bioinformatics field suffer, is they forget to bring this concept into the class. This happens for many reasons:
* It is hard to get good data
* Toy examples can easily teach the same concepts
* Students are often in disparate disciplines

However, this homework aims to introduce you to more "bioinformatic-y" workflows that often are not developed until you hit your lab. While the material that we will be covering is oriented towards bacterial genomics, the concepts should still apply as far as work flow is considered.

---
## Background

> _B. subtilis_ is a Gram-positive bacterium that is often used as a model organism in the study of bacterial chromosome replication. It is also considered to be the best studied Gram-positive bacterial.[$\^1\$](https://wickhamlabs.co.uk/technical-resource-centre/fact-sheet-bacillus-subtilis/)

We will be working with some simulated _B. subtilis_ data today. Some key characteristics of the _B. subtilis_ genome is that it is a 4.13611 megabase (Mb) circular genome with a median GC% of 43.6[$\^2\$](https://www.ncbi.nlm.nih.gov/genome/?term=Bacillus%20subtilis[Organism]&cmd=DetailsSearch).

<img src="https://www.researchgate.net/profile/Antoine_Danchin/publication/24345871/figure/fig1/AS:276900091056136@1443029534678/Circular-representation-of-the-B-subtilis-168-genome-for-several-specific-genome.png" width=600/>

### The Data
A description of the provided data are:
1. `b_subtilis_genome.fa`: A [FASTA format](https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=BlastHelp) file containing the reference sequence for _B. subtilis_
    * A hallmark of the FASTA format is that the sequence header line precedes the sequences and always begins with a '>' character
1. `normal.bam`: A [BAM format](https://samtools.github.io/hts-specs/SAMv1.pdf) file that contains the simulated short reads for a "normal" _B subtilis_ sample
    * This is a very specialized format that needs special libraries to parse. However, just think of it as one read per line
1. `normal.bam.bai`: A BAM index file used for random access
1. `tumor.bam`: A [BAM format](https://samtools.github.io/hts-specs/SAMv1.pdf) file that contains the simulated short reads for a "tumor" _B subtilis_ sample
1. `tumor.bam.bai`: A BAM index file used for random access

The SAM/BAM format can be summarized in this table:</br>
<img src='https://raw.githubusercontent.com/betteridiot/b575f19/master/assets/bam_format.png' width=600 />

<div style="background-color:#ffc680">
  <h3>Important Note</h3>
  <p>You will be using a special Python library for handling this data. This package is called BAMnostic.</p>
    <p><b>Before</b> doing this homework, you will need to install BAMnostic. To do so, go to your terminal and type:<code>conda install -c conda-forge bamnostic</code></p> 
  <p>Consider taking a look at the <a href="https://bamnostic.readthedocs.io/en/latest/?badge=latest">BAMnostic documentation</a> for more information.</p>
</div>

### Methods
The data was simulated using the [Bacillus subtilis subsp. subtilis str. 168](https://support.illumina.com/sequencing/sequencing_software/igenome.html) provided by illumina's iGenomes collection.

* [ART](https://www.niehs.nih.gov/research/resources/software/biostatistics/art/) was used to simulate the short reads (`fastq` files) based on the genome above using known base calling error rates and biases within specified illumina technologies
* [SInC](https://sourceforge.net/projects/sincsimulator/files/?source=navbar) was used to modify the ART reads to simulate SNPS, CNVs, and indels within the reads
* [VarSimLab](https://github.com/NabaviLab/VarSimLab) was used to orchestrate the other technologies and generate the short reads necessary for this assignment
* [bwa](http://bio-bwa.sourceforge.net/) was used to align the reads to the reference genome
* [samtools](http://www.htslib.org/) was used to sort, merge, and index the resultant files

Assuming that all of the above software is installed correctly, I used the following command to generate the data:
>```bash
python Exome_Script.py -use_genome -c 7 -s -snp 10 -l 100 -sam output b_subtilis_genome.fa
```

This means that there are two samples (normal and tumor) of $\approx$ 7x coverage of $\approx$ 100 bp long reads with a SNP rate of 10% across the genome of _B. subtilis_. As this is a cancer cell line simulation workflow, the "tumor" sample should significantly differ from the "normal".

---
## Instructions

This homework is designed to be as close to real genomics research as you can get without the math/stats/research. You are tasked to serially process both the `normal.bam` and `tumor.bam` sample files. For each position on the genome, you will track the number of reads that support that position (`depth`)  for a given sample, the counts of each base observed at that position (`counts`), and the consensus base at that position (`consensus`). The data structure you will be using looks like this:</br>
<img src="https://raw.githubusercontent.com/betteridiot/b575f19/master/assets/hw4_structure.png" align="middle" width=600/>

So, to reiterate, your data structure is:
```python
len(genome_positions) == len_of_genome
type(genome_positions) == list

# Every position will have this data structure
genome_positions[0] = { 
    'normal': {
        'depth': 0,          # number of reads that support this position
        'counts': Counter(), # Count of observed bases at this position
        'consensus': 0       # The most observed base at this position
    },{
    'tumor': {
        'depth': 0,          # number of reads that support this position
        'counts': Counter(), # Count of observed bases at this position
        'consensus': 0       # The most observed base at this position
    }
}
```

Using `bamnostic` you will iterate through the files (`normal.bam` and `tumor.bam`) one read at a time. You will have to perform the following steps:
* Identify the read's starting position against the reference (`read.pos`)
* Using that position:
    * Iterate through the read's sequence (`read.seq`) one letter at a time
    * Keep a count of all observed bases
    * Keep count of number of reads that have overlapped that position
    * Keep count of which base has been observed the most at that position

For example:
```python
>>> print(normal_read1.pos, normal_read1.seq)
20 GTATCCACAGAGGTTATCGACAACATTTTCACATTACCAACCCCTGTGGACAAGGTTTTTTCAACAGGTTGTCCGCTTTGTGGATAAGATTGTGACAACC

>>> print(normal_read1.pos, normal_read1.seq)
28 AGAGGTTATCGACCACATTTTCACATTACCAACCCGTGTGGACAAGGTTTTTTCAACAGGTTGTCCGCTTTGTGGATAAGATTGTGACAACCATTGCAAG

>>> print(genome_positions[28]['normal'])
{'depth': 2, 'counts': Counter({'A': 2}), 'consensus': 'A'}
```

<div style="background-color:#ffc680">
  <h3>Important Note</h3>
    <p>You <b>only</b> need to use <code>read.seq</code> and <code>read.pos</code> to complete this assignment</p>
    <p>You <b>do not</b> have to consider <em>qualities, flags, or CIGAR strings</em> at this time</p>
</div>

### The result

When you have finished processing the files, you will need to produce a second `list` of `tuples` that if and only if the following condition is met:
> More than half of the total reads at that specific position call a different consensus base in the tumor sample versus the normal sample at the same position

The data each of the `tuple`s must contain are:
1. The position of the variant
1. The variant base
1. The reference base
1. The allele frequency of the variant base (counts of variant base calls/total base counts at the given position)

---
## The Coding Contract

You should need to create no less than four (4) functions to finish this assignment:
1. `initialize_genome_list`:
    * Input:
        * genome filename
    * Output:
        * initialized `genome_list`
1. `process_bam`: 
    * Input: 
        * filename to be processed
        * Sample name (`'normal'` or `'tumor'`)
        * `genome_list`
    * Output: Should return the modified `genome_list` given that specific sample
1. `process_read`:
    * Input:
        * `bamnostic.core.AlignedSegment`: This is just the read object type
        * Sample name (`'normal'` or `'tumor'`)
        * `genome_list`
    * Output: Should return the modified `genome_list`
1. `process_results`:
    * Input: Should take `genome_list` as an argument
    * Output: Should return the summarized variants as a `list`

---
## Academic Honor Code
In accordance with Rackham's Academic Misconduct Policy; upon submission of your assignment, you (the student) are indicating acceptance of the following statement:

> “I pledge that this submission is solely my own work.”

As such, the instructors reserve the right to process any and all source code therein contained within the submitted notebooks with source code plagiarism detection software.

Any violations of the this agreement will result in swift, sure, and significant punishment.

---
## Due date
This assignment is due **October 14th, 2019 by Noon (12 PM)**

---
## Submission
> `<uniqname>_hw4.ipynb`

### Example
> `mdsherm_hw4.ipynb`

We will *only* grade the most recent submission of your homework.

---
## Late Policy
Each submission will receive a **10%** penalty per day (up to three days) that the assignment is late.

After that, the student will receive a **0** for the exam.

---
## Good luck and code responsibly!

---

In [20]:
from collections import Counter
import bamnostic as bs

I have set this up so that you do not have to worry about dealing with `bamnostic` directly. You should only have to handle the `read` object from here on out.

In [ ]:
# Initialize your genome_list here
def initialize_genome_list(genome_filename):
    # Do your stuff here
    
    return genome_list

In [ ]:
def process_bam(filename, sample_name, genome_list = None):
    with bs.AlignmentFile(filename) as bam:
        for read in bam:
            # Do your stuff here
            
    return genome_list

In [ ]:
def process_read(read, sample_name, genome_list = None):
    # Do your stuff here
    
    return genome_list

In [ ]:
def process_results(genome_list = None):
    # Do your stuff here
    
    return variant_calls

---
## This last cell should work if all the code above it is run

In [ ]:
# Initialize the list
genome_list = initialize_genome_list('b_subtilis_genome.fa')

# Process all the bam files
for filename in ('normal.bam', 'tumor.bam'):
    genome_list = process_bam(filename, filename.split('.')[0], genome_list)

# Process the results
results = process_results(genome_list)

# Print out the first 10
print(results[:10])